In [17]:
# importing os module
import os
import numpy as np
from PIL import Image
from numpy import asarray

In [18]:
def img_to_array(emotion):
    #first_two = emotion[0:2]
    x = []
    y = []
    #x.append([])
    #x[0].append([])
    folder = "C:/Users/User/Project 4 local/archive/"+emotion
    for count, filename in enumerate(os.listdir(folder)):
        # load the image
        img = Image.open(f"{folder}/{filename}")
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        img_data = asarray(img)
        #img_data = img_data.reshape(-1,1)
        #np.append(X, img_data,axis=0)
        x.append(img_data)
        y.append(filename[0:2])
    return x,y

In [19]:
feature1,target1 = img_to_array("anger")
feature2,target2 = img_to_array("contempt")


In [20]:
feature = feature1 + feature2
len(feature)

1098

In [21]:
feature_arr = np.array(feature)

In [22]:
feature_arr.shape

(1098, 224, 224)

In [23]:
target = target1 + target2
target_arr = np.array(target)
target_arr.shape

(1098,)

In [24]:
X = feature_arr
y= target_arr

In [25]:
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
X_train.shape

(823, 224, 224)

In [27]:
y_train.shape

(823,)

In [28]:
X_train[0, :, :]

array([[21, 21, 21, ..., 45, 45, 45],
       [21, 21, 21, ..., 45, 45, 45],
       [22, 22, 22, ..., 19, 19, 19],
       ...,
       [ 9,  9,  9, ...,  1,  1,  1],
       [ 7,  7,  7, ...,  1,  1,  1],
       [ 7,  7,  7, ...,  1,  1,  1]], dtype=uint8)

In [29]:
# We want to flatten our image of 28x28 pixels to a 1D array of 784 pixels
ndims = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], ndims)
X_test = X_test.reshape(X_test.shape[0], ndims)
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: (823, 50176)
Testing Shape: (275, 50176)


In [45]:
# Dependencies to Visualise the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import tensorflow as tf
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
tf.config.run_functions_eagerly(True)

In [31]:
# Next, we normalise our training data to be between 0 and 1
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
y_train[:200]

array(['an', 'an', 'an', 'an', 'an', 'an', 'co', 'an', 'co', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'co', 'an', 'an', 'an', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an',
       'co', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'co', 'an', 'co',
       'co', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'co', 'co',
       'an', 'co', 'an', 'an', 'an', 'co', 'an', 'co', 'an', 'an', 'co',
       'an', 'an', 'an', 'an', 'an', 'an', 'an', 'co', 'an', 'an', 'an',
       'co', 'an', 'an', 'an', 'an', 'co', 'co', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'an', 'co', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an', 'an',
       'an', 'an', 'an', 'an', 'an', 'co', 'an', 'co', 'co', 'an', 'an',
       'an', 'an', 'an', 'co', 'an', 'an', 'co', 'a

In [17]:
# y_train_reshaped = y_train.reshape(-1, 1) 
# y_test_reshaped = y_test.reshape(-1, 1)

In [33]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# cat = OneHotEncoder()
# y_train_t = y_train_reshaped.T
# y_test_t = y_test_reshaped.T
# y_train_cat = cat.fit_transform(y_train_t).toarray()
# y_test_cat = cat.fit_transform(y_test_t).toarray()


In [34]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [35]:
# We need to convert our target labels (expected values) to categorical data
num_classes = 2
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)
# Original label of `5` is one-hot encoded as `0000010000`
y_train_cat[0]

array([1., 0.], dtype=float32)

In [43]:
X_train.shape

(823, 50176)

In [ ]:
y_train

In [36]:
# Create an empty sequential model
model = Sequential()

In [37]:
# Add the first layer where the input dimensions are the 784 pixel values
# We can also choose our activation function. `relu` is a common
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [38]:
# Add a second hidden layer
model.add(Dense(100, activation='relu'))

In [39]:
# Add our final output layer where the number of nodes 
# corresponds to the number of y labels
model.add(Dense(2, activation='softmax'))

In [40]:
# We can summarise our model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5017700   
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 5,028,002
Trainable params: 5,028,002
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [46]:
# Fit (train) the model
model.fit(
    X_train,
    y_train_cat,
    epochs=10,
    shuffle=True,
    verbose=2
)

c:\Users\User\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\data\ops\structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
26/26 - 6s - loss: 3.6247 - accuracy: 0.6987 - 6s/epoch - 225ms/step
Epoch 2/10
26/26 - 5s - loss: 1.5104 - accuracy: 0.6841 - 5s/epoch - 203ms/step
Epoch 3/10
26/26 - 5s - loss: 2.3032 - accuracy: 0.6768 - 5s/epoch - 211ms/step
Epoch 4/10
26/26 - 5s - loss: 1.1328 - accuracy: 0.7193 - 5s/epoch - 199ms/step
Epoch 5/10
26/26 - 5s - loss: 0.8709 - accuracy: 0.7254 - 5s/epoch - 207ms/step
Epoch 6/10
26/26 - 7s - loss: 0.4919 - accuracy: 0.8068 - 7s/epoch - 260ms/step
Epoch 7/10
26/26 - 6s - loss: 0.7405 - accuracy: 0.7242 - 6s/epoch - 246ms/step
Epoch 8/10
26/26 - 7s - loss: 0.6000 - accuracy: 0.7655 - 7s/epoch - 252ms/step
Epoch 9/10
26/26 - 7s - loss: 0.4529 - accuracy: 0.7910 - 7s/epoch - 264ms/step
Epoch 10/10
26/26 - 6s - loss: 0.4549 - accuracy: 0.8032 - 6s/epoch - 239ms/step
